# The network energy intensity of video streaming over Wi-Fi and 4G

**Authors:** David Mytton, Iain Staffell, Malte Jansen.

**Institution:** Centre for Environmental Policy, Imperial College London, London, SW7 1NE, UK.

**Correspondence:** <david@davidmytton.co.uk>.

## Summary

> Between 2010-2018 internet traffic grew ten-fold and is expected to double again by 2022. With video streaming making up 60% of that traffic, 65% on mobile, it is important to understand its energy consumption. Here we provide updated figures for the energy consumption of different sections of the internet by using observations from volunteers of home internet routing over Wi-Fi and 4G connections, then use survey and industry estimates to calculate the network energy intensity of video streaming. We estimate the 2019 network use-stage energy footprint of the UK’s 39 billion hours of video streaming at 4.2 TWh, or 1.3% of total electricity generation. We show that the network energy intensity of video streaming over Wi-Fi is 0.091 kWh/hour, compared to 0.207 kWh/hour over 4G mobile internet.

## This notebook

This notebook walks through the methodology and calculations as described in the accompanying paper.

#### Imports

In [ ]:
%pip install -r requirements.txt

import numpy as np
import pandas as pd
import pint
from pint import UnitRegistry

ureg = UnitRegistry()

## Internet video traffic

We use YouTube as a case study because it is the largest video streaming service by traffic so has the most data available. We use various sources to calculate the total amount of time, data volume, and assuming YouTube is representative of all video streaming, scale those numbers to calculate the use-stage network energy consumption of video streaming in general.

We therefore start with an assessment of YouTube.

### YouTube watch time

YouTube does not publish statistics so we use figures from the advertising industry<sup>1</sup>, Ofcom<sup>2</sup>, and Netflix<sup>3</sup>.

In [ ]:
# Dec 2019 UK YouTube minutes (mobile devices)
youtubeTimeDecDevices = 32099000000 * ureg.minutes  # Source [1] (Pinpoint: pg 30)

# What % watch YouTube on a mobile devices
youtubeWatchPercentageMobileDevices = 0.73  # 73% Source [2] (Pinpoint: Figure 4.10, pg116)

# What % watch YouTube via a mobile network
youtubeWatchPercentageMobileNetworks = 0.25  # 25% Source [3]

Using the total annual watch minutes, the total time watched on mobile devices and the total time watched on mobile networks, we can calculate the total time spent on Wi-Fi and non-mobile devices.

In [ ]:
# Assume Dec 2019 figure is representative of the full year
youtubeTimeAnnualDevices = youtubeTimeDecDevices * 12

# Annual UK YouTube minutes (all networks, all devices)
youtubeTimeAnnualAllNetworksAllDevices = youtubeTimeAnnualDevices / youtubeWatchPercentageMobileDevices
print(f'Annual UK YouTube minutes (all networks, all devices): {youtubeTimeAnnualAllNetworksAllDevices}:,.0f')

print(f'Annual UK YouTube minutes (mobile devices): {youtubeTimeAnnualAllNetworksAllDevices:,.0f}')

# Annual UK YouTube minutes (mobile networks)
youtubeTimeAnnualMobileNetworks = youtubeWatchPercentageMobileNetworks * youtubeTimeAnnualAllNetworksAllDevices
print(f'Annual UK YouTube minutes (mobile networks): {youtubeTimeAnnualMobileNetworks:,.0f}')

# Annual UK YouTube minutes (Wi-fi networks, all devices)
youtubeTimeAnnualWifiAllDevices = youtubeTimeAnnualAllNetworksAllDevices - youtubeTimeAnnualMobileNetworks
print(f'Annual UK YouTube minutes (Wi-Fi networks, all devices): {youtubeTimeAnnualWifiAllDevices:,.0f}')

Annual UK YouTube minutes (all networks, all devices): 527654794520.548 minute:,.0f
Annual UK YouTube minutes (mobile devices): 527,654,794,521 minute
Annual UK YouTube minutes (mobile networks): 131,913,698,630 minute
Annual UK YouTube minutes (Wi-Fi networks, all devices): 395,741,095,890 minute


### YouTube data volume

The amount of data transmitted during a single YouTube video streaming session varies based on factors such as device type, screen size, video resolution, framerate, bit rate, network speed and which formats the video was encoded into. 

For each video quality setting, we take the mean average of the range of values reported in an observational assessment of 1 hour of streaming<sup>4</sup>. In this assessment, data volume was recorded using three separate tools: Android's built-in data monitoring, Google's Datally app and the GlassWire data monitoring app.

In [ ]:
dataVolume = pd.DataFrame({
    480: [0.48 * ureg.gigabyte, 0.66 * ureg.gigabyte], # 480p SD [min, max]
    720: [1.2 * ureg.gigabyte, 2.7 * ureg.gigabyte], # 720p HD [min ,max]
    1080: [2.5 * ureg.gigabyte, 4.1 * ureg.gigabyte], # 1080p FHD [min ,max]
    1440: [2.7 * ureg.gigabyte, 8.1 * ureg.gigabyte], # 1440p QHD [min ,max]
    2160: [5.5 * ureg.gigabyte, 23.0 * ureg.gigabyte], # 2160p UHD 4k [min ,max]
})

### Video traffic

We need to know how much traffic is from YouTube and video in general for both mobile and non-mobile.

In [ ]:
internetTrafficYouTube = 0.118 # 11.8% Source [5] (Pinpoint: pg12)
internetTrafficVideo = 0.60 # 60% Source [5] (Pinpoint: pg6)
mobileTrafficYouTube = 0.271 # 27.1% Source [6] (Pinpoint: pg9)
mobileTrafficVideo = 0.655 # 65.5% Source [6] (Pinpoint: pg5)

## Video streaming time and data

Using the figures above, we calculate the total annual time spent and associated data volume for YouTube.

We assume mobile traffic is all 4G and non-mobile traffic is all Wi-Fi because 97% of UK premises have access to decent fixed (at least 10 Mbit/s) and 4G (at least 2 Mbit/s) services, 91% of geographic areas of the UK are able to receive 4G data service from at least one operator, and 4G carries 90% of UK mobile data traffic<sup>7</sup>. 3G connectivity is excluded because it carries only a small proportion of data traffic.

We assume the default video quality is 720p High Definition (HD) because during the 2020 Coronavirus pandemic, the default was reduced to 480p Standard Definition (SD)<sup>8</sup>. The reference year for this assessment is 2019.

In [ ]:
# 720p HD video quality
videoQuality = 720

### YouTube

In [ ]:
# Tidy up the calculations into an easily readable data frame
youtube = pd.DataFrame({
    'Time': {
        '4G': youtubeTimeAnnualMobileNetworks,
        'Wi-Fi': youtubeTimeAnnualWifiAllDevices,
        'Total': youtubeTimeAnnualMobileNetworks + youtubeTimeAnnualWifiAllDevices,
    },
    'Data': {
        '4G': youtubeTimeAnnualMobileNetworks.to('hours').magnitude * dataVolume[videoQuality].values.mean(),
        'Wi-Fi': youtubeTimeAnnualWifiAllDevices.to('hours').magnitude * dataVolume[videoQuality].values.mean(),
        'Total': (youtubeTimeAnnualMobileNetworks.to('hours').magnitude * dataVolume[videoQuality].values.mean()) + (youtubeTimeAnnualWifiAllDevices.to('hours').magnitude * dataVolume[videoQuality].values.mean()),
    }
})

# Format nicely for display
youtubeDisplay = pd.DataFrame({
    'Time': {
        '4G': f'{youtube["Time"]["4G"].to("hour"):,.0f} ({youtube["Time"]["4G"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Wi-Fi': f'{youtube["Time"]["Wi-Fi"].to("hour"):,.0f} ({youtube["Time"]["Wi-Fi"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Total': f'{youtube["Time"]["Total"].to("hour"):,.0f} ({youtube["Time"]["Total"].to("hour").to_compact().magnitude:,.1f} bn)',

    },
    'Data': {
        '4G': f'{youtube["Data"]["4G"].to("gigabyte"):,.0f~H} ({youtube["Data"]["4G"].to("exabyte"):,.1f~H})',
        'Wi-Fi': f'{youtube["Data"]["Wi-Fi"].to("gigabyte"):,.0f~H} ({youtube["Data"]["Wi-Fi"].to("exabyte"):,.1f~H})',
        'Total': f'{youtube["Data"]["Total"].to("gigabyte"):,.0f~H} ({youtube["Data"]["Total"].to("exabyte"):,.1f~H})',

    }
})
youtubeDisplay

,Time,Data
4G,"2,198,561,644 hour (2.2 bn)","4,287,195,205 GB (4.3 EB)"
Wi-Fi,"6,595,684,932 hour (6.6 bn)","12,861,585,616 GB (12.9 EB)"
Total,"8,794,246,575 hour (8.8 bn)","17,148,780,822 GB (17.1 EB)"


#### Validation by comparison

An Ofcom survey<sup>2</sup> reported the number of unique visitors to YouTube out of the UK population and how much time they spent on it per day in 2019. From this we can calculate the total viewing time and compare to our figure.

In [ ]:
# YouTube daily visitors
ofcomYoutubeDailyVisitors = 41970000 # Source [2] (Pinpoint: Figure 4.6, pg111)

# YouTube average daily time spent
ofcomYoutubeAverageDailyTimeSpent = 35 * ureg.minutes # Source [2] (Pinpoint: Figure 4.8, pg 114)

# Annual UK YouTube time spent
ofcomYoutubeAnnualTimeSpent = ((ofcomYoutubeAverageDailyTimeSpent * 365) * ofcomYoutubeDailyVisitors)
print(f'Ofcom survey: {ofcomYoutubeAnnualTimeSpent.to("hours").magnitude:,.0f} ({ofcomYoutubeAnnualTimeSpent.to("hours").to_compact().magnitude:,.1f}bn) hours')
print(f'Our calculation: {youtube["Time"]["Total"].to("hours").magnitude:,.0f} ({youtube["Time"]["Total"].to("hours").to_compact().magnitude:,.1f}bn) hours')

# % difference
difference = ((youtube['Time']['Total'] - ofcomYoutubeAnnualTimeSpent) / youtube['Time']['Total']) * 100
print(f'Difference: {difference.magnitude:,.1f}%')

Ofcom survey: 8,936,112,500 (8.9bn) hours
Our calculation: 8,794,246,575 (8.8bn) hours
Difference: -1.6%


### All UK video streaming

Given that YouTube is a certain percentage of all internet traffic, and we know what percentage of traffic is video streaming, if we assume that YouTube is representative of all video streaming we can extrapolate for all UK video streaming.

In [ ]:
# Extrapolate to all video streaming
allUKVideo4GData = (youtube['Data']['4G'] /
                    mobileTrafficYouTube) * mobileTrafficVideo
allUKVideoWiFiData = (youtube['Data']['Wi-Fi'] /
                      internetTrafficYouTube) * internetTrafficVideo
allUKVideo4GTime = (allUKVideo4GData /
                    dataVolume[videoQuality].values.mean()) * ureg.hours
allUKVideoWiFiTime = (allUKVideoWiFiData /
                      dataVolume[videoQuality].values.mean()) * ureg.hours

# Create a dataframe for easy analysis
allUKVideo = pd.DataFrame({
    'Time': {
        '4G': allUKVideo4GTime,
        'Wi-Fi': allUKVideoWiFiTime,
        'Total': allUKVideo4GTime + allUKVideoWiFiTime,
    },
    'Data': {
        '4G': allUKVideo4GData,
        'Wi-Fi': allUKVideoWiFiData,
        'Total': allUKVideo4GData + allUKVideoWiFiData,
    }
})

# Format nicely for display
allUKVideoDisplay = pd.DataFrame({
    'Time': {
        '4G': f'{allUKVideo["Time"]["4G"].to("hour"):,.0f} ({allUKVideo["Time"]["4G"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Wi-Fi': f'{allUKVideo["Time"]["Wi-Fi"].to("hour"):,.0f} ({allUKVideo["Time"]["Wi-Fi"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Total': f'{allUKVideo["Time"]["Total"].to("hour"):,.0f} ({allUKVideo["Time"]["Total"].to("hour").to_compact().magnitude:,.1f} bn)',

    },
    'Data': {
        '4G': f'{allUKVideo["Data"]["4G"].to("gigabyte"):,.0f~H} ({allUKVideo["Data"]["4G"].to("exabyte"):,.1f~H})',
        'Wi-Fi': f'{allUKVideo["Data"]["Wi-Fi"].to("gigabyte"):,.0f~H} ({allUKVideo["Data"]["Wi-Fi"].to("exabyte"):,.1f~H})',
        'Total': f'{allUKVideo["Data"]["Total"].to("gigabyte"):,.0f~H} ({allUKVideo["Data"]["Total"].to("exabyte"):,.1f~H})',
    }
})
allUKVideoDisplay

,Time,Data
4G,"5,313,866,704 hour (5.3 bn)","10,362,040,072 GB (10.4 EB)"
Wi-Fi,"33,537,381,008 hour (33.5 bn)","65,397,892,965 GB (65.4 EB)"
Total,"38,851,247,711 hour (38.9 bn)","75,759,933,037 GB (75.8 EB)"


#### Validation by comparison

An Ofcom survey<sup>9</sup> reported the total number of video minutes watched per day per person and the proportion of that which was not broadcast content i.e. online video streaming vs broadcast TV. Based on the UK population<sup>10</sup>, we can calculate the total viewing time and compare to our figure.

In [ ]:
ofcomTotalTimeOnlinePerPerson = 209 * ureg.minutes # Source [2] (Pinpoint: Paragraph, pg 9)

ofcomTotalVideoTimeDaily = 294  * ureg.minutes # Source [9] (Pinpoint: Figure 1.4, pg 16)
ofcomPercentageNonBroadcast = 0.31 # Source [9] (Pinpoint: Figure 1.4, pg 16)
ukPopulation = 66400000 # Source [10]

ofcomTotalVideoStreamingTime = (ofcomTotalVideoTimeDaily * ofcomPercentageNonBroadcast * ukPopulation * 365)

print(f'Ofcom survey: {ofcomTotalVideoStreamingTime.to("hours").magnitude:,.0f} ({ofcomTotalVideoStreamingTime.to("hours").to_compact().magnitude:,.1f}bn) hours')
print(f'Our calculation: {allUKVideo["Time"]["Total"].to("hours").magnitude:,.0f} ({allUKVideo["Time"]["Total"].to("hours").to_compact().magnitude:,.1f}bn) hours')

# % difference
difference = ((allUKVideo['Time']['Total'] -
              ofcomTotalVideoStreamingTime) / allUKVideo['Time']['Total']) * 100
print(f'Difference: {difference.magnitude:,.1f}%')

Ofcom survey: 36,814,484,000 (36.8bn) hours
Our calculation: 38,851,247,711 (38.9bn) hours
Difference: 5.2%


## Traceroute sample analysis

116 Scamper traceroute samples were returned by 29 participants (see main paper for background and recruitment methods). The individual samples are provided in `/traceroute-samples/samples/`. An aggregated CSV is provided in `/traceroute-samples/traceroute-samples.csv` with network ownership metadata returned by [IPInfo](https://ipinfo.io). Two samples returned anomalous results where zero hops were reported - these are excluded as `NaN`. 

See `/traceroute-samples/aggregate-samples.ipynb` for the aggregation code.

In [ ]:
traceroutes = pd.read_csv('../traceroute-samples/traceroute-samples.csv')

# ASN Names
googleASN = 'AS15169 Google LLC'
facebookASN = 'AS32934 Facebook, Inc.'

# Destination ASN = 4G
traceroutes4G = traceroutes.query('Connection == "4g"')
traceroutesWiFi = traceroutes.query('Connection == "wifi"')

# Destination ASN = ISP
traceroutesISPAll = traceroutes.query(
    '`Destination ASN` != @googleASN and `Destination ASN` != @facebookASN')
traceroutesISP4G = traceroutesISPAll.query('Connection == "4g"')
traceroutesISPWiFi = traceroutesISPAll.query('Connection == "wifi"')

# Destination ASN = Facebook or Google
traceroutesFBGOOGAll = traceroutes.query(
    '`Destination ASN` == @googleASN or `Destination ASN` == @facebookASN')
traceroutesFBGOOG4G = traceroutesFBGOOGAll.query('Connection == "4g"')
traceroutesFBGOOGWiFi = traceroutesFBGOOGAll.query('Connection == "wifi"')

### Hop counts

Calculate the mean hop count and standard deviation based on the owner of the destination network. The ISP owning the destination network indicates a caching device is deployed within the network.

In [ ]:
hopCounts = pd.DataFrame({
    'Count': {  # Count total number of samples
        'All': traceroutes['Participant City'].count(),
        'Connection: 4G': traceroutes4G['Participant City'].count(),
        'Connection: Wi-Fi': traceroutesWiFi['Participant City'].count(),
        'Destination - ISP: All': traceroutesISPAll['Participant City'].count(),
        'Destination - ISP: 4G': traceroutesISP4G['Participant City'].count(),
        'Destination - ISP: Wi-Fi': traceroutesISPWiFi['Participant City'].count(),
        'Destination - FB or GOOG: All': traceroutesFBGOOGAll['Participant City'].count(),
        'Destination - FB or GOOG: 4G': traceroutesFBGOOG4G['Participant City'].count(),
        'Destination - FB or GOOG: Wi-Fi': traceroutesFBGOOGWiFi['Participant City'].count(),
    },
    'Mean': {  #  Mean hop count, excluding anomalous results
        'All': traceroutes['Trace Hop Count'].mean(skipna=True),
        'Connection: 4G': traceroutes4G['Trace Hop Count'].mean(skipna=True),
        'Connection: Wi-Fi': traceroutesWiFi['Trace Hop Count'].mean(skipna=True),
        'Destination - ISP: All': traceroutesISPAll['Trace Hop Count'].mean(skipna=True),
        'Destination - ISP: 4G': traceroutesISP4G['Trace Hop Count'].mean(skipna=True),
        'Destination - ISP: Wi-Fi': traceroutesISPWiFi['Trace Hop Count'].mean(skipna=True),
        'Destination - FB or GOOG: All': traceroutesFBGOOGAll['Trace Hop Count'].mean(skipna=True),
        'Destination - FB or GOOG: 4G': traceroutesFBGOOG4G['Trace Hop Count'].mean(skipna=True),
        'Destination - FB or GOOG: Wi-Fi': traceroutesFBGOOGWiFi['Trace Hop Count'].mean(skipna=True),
    },
    'StdDev': {  #  Hop count standard deviation, excluding anomalous results
        'All': traceroutes['Trace Hop Count'].std(skipna=True),
        'Connection: 4G': traceroutes4G['Trace Hop Count'].std(skipna=True),
        'Connection: Wi-Fi': traceroutesWiFi['Trace Hop Count'].std(skipna=True),
        'Destination - ISP: All': traceroutesISPAll['Trace Hop Count'].std(skipna=True),
        'Destination - ISP: 4G': traceroutesISP4G['Trace Hop Count'].std(skipna=True),
        'Destination - ISP: Wi-Fi': traceroutesISPWiFi['Trace Hop Count'].std(skipna=True),
        'Destination - FB or GOOG: All': traceroutesFBGOOGAll['Trace Hop Count'].std(skipna=True),
        'Destination - FB or GOOG: 4G': traceroutesFBGOOG4G['Trace Hop Count'].std(skipna=True),
        'Destination - FB or GOOG: Wi-Fi': traceroutesFBGOOGWiFi['Trace Hop Count'].std(skipna=True),
    }
})

hopCounts

,Count,Mean,StdDev
All,116,7.412281,4.244333
Connection: 4G,58,7.362069,5.000696
Connection: Wi-Fi,58,7.464286,3.330107
Destination - ISP: All,54,6.096154,2.443556
Destination - ISP: 4G,20,7.600000,2.722228
Destination - ISP: Wi-Fi,34,5.156250,1.705955
Destination - FB or GOOG: All,62,8.516129,5.065932
Destination - FB or GOOG: 4G,38,7.236842,5.888304
Destination - FB or GOOG: Wi-Fi,24,10.541667,2.302724


Mean and standard deviation for hop count where the destination network belongs to the ISP:

In [ ]:
print(f'{hopCounts["Mean"]["Destination - ISP: All"]:.1f} ± {hopCounts["StdDev"]["Destination - ISP: All"]:.1f} (from n = {hopCounts["Count"]["Destination - ISP: All"]:.0f} samples)')

6.1 ± 2.4 (from n = 54 samples)


Mean and standard deviation for hop count where the destination network belongs to Facebook or Google:

In [ ]:
print(f'{hopCounts["Mean"]["Destination - FB or GOOG: All"]:.1f} ± {hopCounts["StdDev"]["Destination - FB or GOOG: All"]:.1f} (from n = {hopCounts["Count"]["Destination - FB or GOOG: All"]:.0f} samples)')

8.5 ± 5.1 (from n = 62 samples)


### Apportion by connection type

We want to apportion the traffic so it is representative of the routing revealed by the traceroutes. This means we need to calculate what percentage of traffic terminates at the ISP for 4G and Wi-Fi.

In [ ]:
trafficPercentISP4G = hopCounts['Count']['Destination - ISP: 4G'] / hopCounts['Count']['Connection: 4G']
trafficPercentFBGOOG4G = hopCounts['Count']['Destination - FB or GOOG: 4G'] / hopCounts['Count']['Connection: 4G']
print(f'4G traffic: {trafficPercentISP4G:.0%} terminates at the ISP and {trafficPercentFBGOOG4G:.0%} terminates at Facebook or Google.')

trafficPercentISPWiFi = hopCounts['Count']['Destination - ISP: Wi-Fi'] / hopCounts['Count']['Connection: Wi-Fi']
trafficPercentFBGOOGWiFi = hopCounts['Count']['Destination - FB or GOOG: Wi-Fi'] / hopCounts['Count']['Connection: Wi-Fi']
print(f'Wi-Fi traffic: {trafficPercentISPWiFi:.0%} terminates at the ISP and {trafficPercentFBGOOGWiFi:.0%} terminates at Facebook or Google.')

4G traffic: 34% terminates at the ISP and 66% terminates at Facebook or Google.
Wi-Fi traffic: 59% terminates at the ISP and 41% terminates at Facebook or Google.


## Internet energy

The internet consists of multiple sections which connect the end-user device to the data center, so we must assess them each separately.

The end-user device is included for 4G connections because radio transmission is built into the device used for watching and therefore cannot be excluded. Over Wi-Fi, the many different types of end-user device mean the energy consumption is highly variable and would therefore confuse any estimates of use-stage network energy because the device is not a logical part of the network. The end-user device is therefore excluded for Wi-Fi connections.

The internet is split into three sections: Edge, Metro, and Core, which are made up of multiple components such as switches, routers, and fiber multiplexers, each with different deployment characteristics such as number of switches/routers, and utilization values.

The data center, caching nodes and end-user device on Wi-Fi networks are excluded, see System boundaries in the main paper for reasoning.

Here we calculate the use-stage network transmission energy intensity of each section.

#### Units

Define new `pint` units:

In [ ]:
ureg.define('watthour_hour = watthour / hour = Wh/hour')
ureg.define('kilowatthour_hour = kilowatthour / hour = kWh/hour')
ureg.define('kilowatthour_gb = kilowatthour / gigabyte = kWh/GB')
ureg.define('joules_gigabit = joules / gigabit = J/Gb')
ureg.define('kilowatthours_gigabyte = kilowatt hours / gigabyte = kWh/GB')

### CPE & AN - Wi-Fi

For Wi-Fi, we follow the approach in Coroama et al.<sup>11</sup> and Schien et al.<sup>12</sup> with calculations from the "Internet video traffic" section above and updated PUE figures from Masanet et al.<sup>13</sup>.

#### CPE

In [ ]:
# Total time equipment is on 
# (Pinpoint: pg16 in [11], citing Table 7-1 in [14])
cpeWifiTOn = 1440 * ureg.minutes

# Total time in which the router is in use
# Calculated based on the total amount of time spent online from Ofcom survey [2]
# Apportioned based on the total amount of time spent video streaming on Wi-Fi
# vs mobile.
cpeWifiTUse = ofcomTotalTimeOnlinePerPerson - \
    (ofcomTotalTimeOnlinePerPerson * mobileTrafficYouTube)

# Total idle time when the router is on but not used
cpeWifiTIdle = cpeWifiTOn - cpeWifiTUse

# Power of the router
# (Pinpoint: pg15 in [11])
cpeWifiRouterPower = 8 * ureg.watts

#### AN

In [ ]:
# Redundancy
# (Pinpoint: pg12 in [11])
anWifiR = 2

# Power of the access network devices
# (Pinpoint: pg15 in [11], citing pg5 (804) in [12].)
anWifiPower = 2 * ureg.watts

# Number of users connected to access devices
# (Pinpoint: pg14 in [11])
anWifiUsers = 1

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional 
# Data center PUE Western Europe 2019.)
anWifiPUE = 1.99

#### CPE & AN

In [ ]:
iCPEANWifi = (1 + cpeWifiTIdle / cpeWifiTUse) * cpeWifiRouterPower + \
    ((anWifiPower / anWifiUsers) * anWifiR) * anWifiPUE
iCPEANWifi = (iCPEANWifi.magnitude) * ureg.watthour_hour
iCPEANWifi.to('kilowatthour_hour')

0.08356990017130368 <Unit('kilowatthour_hour')>

### CPE & AN - 4G

For 4G AN, Pihkola et al.<sup>15</sup> is the most up to date peer-reviewed figure – 0.1 kWh/GB. 

A figure for 4G CPE is more difficult to calculate because the equivalent of the Wi-Fi router is embedded in the device used for watching, so the end-user device must be included. There are many different types of phone or tablet with varying battery capacities. A simplified approach was taken by using data from Apple for the iPhone 11 Pro<sup>16,17</sup>. Apple reports that a battery charged to 100% will support 11 hours of streamed video<sup>18</sup>. We assume assuming battery consumption is linear and that 100% of the battery is depleted after 11 hours.

#### CPE

In [ ]:
iPhoneCapacity = 11.67 * ureg.watthours
streamingHours = 11 * ureg.hours

cpe4G = iPhoneCapacity / streamingHours
cpe4G

1.0609090909090908 <Unit('watt_hour / hour')>

#### AN

In [ ]:
an4G = 0.1 * ureg.kilowatthour_gb # Source [15]
an4G

0.1 <Unit('kilowatthour_gb')>

#### CPE & AN

Unlike with Wi-Fi, we can't combine 4G CPE & AN here because the units are different, so this is done later.

### Edge, Metro, Core

For the sections of the internet - Edge, Metro and Core - we follow the approach in Schien et al.<sup>12</sup> and [associated implementation details](https://nbviewer.jupyter.org/gist/dschien/24bbb049ba9be347fc22), with some modifications.

The assumption of fiber Access Network connectivity used by Schien et al.<sup>12</sup> in 2015 was criticized by Aslan et al.<sup>19</sup>. However, we assume this is now appropriate for the UK in 2019 where 95% of premises have access to superfast (>30Mbit/s) and 53% have access to ultrafast (300Mbit/s) internet connectivity<sup>7</sup>, both of which use fiber-based technologies such as Fiber to the Cabinet (FTTC).

#### Number of routers

Schien et al.<sup>12</sup> assumes 6 routers in total with 4 routers described as being in the "long haul" network, which we assume to mean "Core" network. Instead, we use our traceroute results (above) and allocate across Metro and Core in the same proportion, assuming Edge always has 1 router. We follow the approach of Schien et al.<sup>12</sup> where the number of routers in a route of `n` hops is `n + 1`.

In [ ]:
# Original number of routers from [12]
# (Pinpoint: pg7 in [12])
originalRouters = {}
originalRouters['edge'] = 1
originalRouters['metro'] = 1
originalRouters['core'] = 4
originalRouters['total'] = originalRouters['edge'] + originalRouters['metro'] + originalRouters['core']

# Adjusted number based on traceroute results
adjustedRouters = {}
adjustedRouters['edge'] = 1 # Always 1
adjustedRouters['metro'] = originalRouters['metro'] / originalRouters['total'] * (traceroutes['Trace Hop Count'].mean(skipna=True) + 1)
adjustedRouters['core'] = originalRouters['core'] / originalRouters['total'] * (traceroutes['Trace Hop Count'].mean(skipna=True) + 1)
adjustedRouters['total'] = adjustedRouters['edge'] + adjustedRouters['metro'] + adjustedRouters['core']
adjustedRouters

{'edge': 1,
 'metro': 1.402046783625731,
 'core': 5.608187134502924,
 'total': 8.010233918128655}

#### Edge

In [ ]:
# Redundancy
# (Pinpoint: pg7 in [12])
edgeR = 2

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional 
# Data center PUE Western Europe 2019.)
edgePUE = 1.99

# Edge overcapacity factor (% utilisation)
# This is not provided Schien et al. 27 so 
#we assume the value is the same as the Metro network
# (Pinpoint: pg7 in [12])
edgeOvercapacity = 6.67

# Edge switch energy intensity
# (Pinpoint: pg6 in [12])
edgeSwitchI = 8 * ureg.joules_gigabit

# Edge router energy intensity
# (Pinpoint: pg6 in [12])
edgeRouterI = 64 * ureg.joules_gigabit

# Energy intensity of edge
iEdge = edgeR * edgePUE * edgeOvercapacity * (edgeSwitchI + edgeRouterI)
iEdge

1911.3552 <Unit('joules_gigabit')>

#### Metro

In [ ]:
# Redundancy
# (Pinpoint: pg7 in [12])
metroR = 2

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional
# Data center PUE Western Europe 2019.)
metroPUE = 1.99

# Metro overcapacity factor (% utilisation)
# (Pinpoint: pg7 in [12])
metroOvercapacity = 6.67

# Number of metro routers
metroRouters = adjustedRouters['metro']

# Energy intensity per metro router
# (Pinpoint: pg7 in [12])
metroRouterI = 39 * ureg.joules_gigabit

# Energy intensity of metro transmission
# (Pinpoint: pg7 in [12] - "We assume a cumulative, nominal energy intensity of
# the optical transmission system to vary between 230, 147 and 316 J/Gb
# (average, 25th, 75th percentile) for metro"
metroTransmissionI = 230 * ureg.joules_gigabit

# Energy intensity of metro
iMetro = metroPUE * metroOvercapacity * \
    (metroR * metroRouters * metroRouterI + metroTransmissionI)
iMetro

4504.422430701754 <Unit('joules_gigabit')>

#### Core

In [ ]:
# Redundancy
# (Pinpoint: pg7 in [12])
coreR = 2

# PUE of the telecoms site
# (Pinpoint: [13] Supplementary Material, Sheet "Regional PUE" - Traditional 
# Data center PUE Western Europe 2019.)
corePUE = 1.99

# Metro overcapacity factor (% utilisation)
# (Pinpoint: pg7 in [12])
coreOvercapacity = 3.03

# Number of core routers
coreRouters = adjustedRouters['core']

# Energy intensity per core router
# (Pinpoint: pg7 in [12])
coreRouterI = 26.7 * ureg.joules_gigabit

# Energy intensity of core transmission
# (Pinpoint: pg7 in [12] - ""We assume a cumulative, nominal energy intensity 
# of the optical transmission system to vary between  1593, 893 and 2292 J/Gb 
# (average, 25th, 75th percentile) for core networks""
coreTransmissionI = 1593 * ureg.joules_gigabit

# Energy intensity of core
iCore = corePUE * coreOvercapacity * (coreR * coreRouters * coreRouterI + coreTransmissionI)
iCore

11411.069730526313 <Unit('joules_gigabit')>

### Internet

The equipment energy intensity figures used above are taken from Schien et al.<sup>12</sup>, however these are from 2014. The industry suffers from a lack of published data about equipment energy intensity, and it was not possible to find more recent numbers. We therefore apply an efficiency improvement adjustment to the energy intensity for each network component. Future work could survey ISPs to inventory deployed equipment, take energy measurements and produce more accurate data. In lieu of such data, we apply a reduction of 79.82% to the energy intensity based on the expected decrease demonstrated by Aslan et al.<sup>19</sup>.

In [ ]:
# Decrease expected from 2014-2019
expectedDecrease = -0.7982

# Calculate decrease
iEdge2019 = iEdge * (1 + expectedDecrease)
iMetro2019 = iMetro * (1 + expectedDecrease)
iCore2019 = iCore * (1 + expectedDecrease)

iInternet = {
    'edge': iEdge2019.to('kilowatthours_gigabyte'),
    'metro': iMetro2019.to('kilowatthours_gigabyte'),
    'core': iCore2019.to('kilowatthours_gigabyte'),
}
iInternet['total'] = iInternet['edge'] + iInternet['metro'] + iInternet['core']

print(f'Edge: {iInternet["edge"]:.4f~H}')
print(f'Metro: {iInternet["metro"]:.4f~H}')
print(f'Core: {iInternet["core"]:.4f~H}')
print(f'Total: {iInternet["total"]:.4f~H}')

Edge: 0.0009 kWh/GB
Metro: 0.0020 kWh/GB
Core: 0.0051 kWh/GB
Total: 0.0080 kWh/GB


## Video streaming energy

Using the above values, we can now calculate the total use-stage network energy consumption for all video streaming in the UK for 2019.

### CPE & AN

All traffic flows through the CPE & AN, so we calculate the total energy consumption for this section.

In [ ]:
cpeAN4G = (an4G * allUKVideo['Data']['4G']) + \
    (cpe4G * allUKVideo['Time']['4G'])
cpeANWiFi = iCPEANWifi * allUKVideo['Time']['Wi-Fi']
print(f'CPE & AN - 4G: {cpeAN4G.to("gigawatt hours").magnitude:,.0f} GWh')
print(f'CPE & AN - Wi-Fi: {cpeANWiFi.to("gigawatt hours").magnitude:,.0f} GWh')

cpeANTotal = cpeAN4G + cpeANWiFi
print(f'CPE & AN: {cpeANTotal.to("gigawatt hours").magnitude:,.0f} GWh')

CPE & AN - 4G: 1,042 GWh
CPE & AN - Wi-Fi: 2,803 GWh
CPE & AN: 3,845 GWh


### Caching nodes

The traceroute samples reveal that network traffic can have two destination networks:

1. A destination network owned by the ISP. We assume this means the ISP has a caching device deployed within their network and so traffic traverses the CPE & AN and Edge network sections.
2. A destination network owned by the content provider (Google or Facebook in our tests). We assume this means traffic traverses all sections of the network - CPE & AN, Edge, Metro and Core.

We therefore apportion traffic based on the percentages above.

In [ ]:
trafficDestination = pd.DataFrame({
    '4G': {
        'Traffic to ISP edge cache': trafficPercentISP4G * allUKVideo['Data']['4G'],
        'Traffic to content provider': trafficPercentFBGOOG4G * allUKVideo['Data']['4G'],
    },
    'Wi-Fi': {
        'Traffic to ISP edge cache': trafficPercentISPWiFi * allUKVideo['Data']['Wi-Fi'],
        'Traffic to content provider': trafficPercentFBGOOGWiFi * allUKVideo['Data']['Wi-Fi'],
    }
})
trafficDestination

,4G,Wi-Fi
Traffic to ISP edge cache,3573117266.3049264 gigabyte,38336695875.99979 gigabyte
Traffic to content provider,6788922805.97936 gigabyte,27061197088.941025 gigabyte


### Edge, Metro, Core

We now calculate the total energy consumption of the three sections of the internet.

In [ ]:
internetEnergy = pd.DataFrame({
    '4G': {
        'CPE & AN': cpeAN4G,
        # All traffic goes to the edge
        'Edge': allUKVideo['Data']['4G'] * iInternet['edge'],
        'Metro': trafficDestination['4G']['Traffic to content provider'] * iInternet['metro'],
        'Core': trafficDestination['4G']['Traffic to content provider'] * iInternet['core'],
    },
    'Wi-Fi': {
        'CPE & AN': cpeANWiFi,
        # All traffic goes to the edge
        'Edge': allUKVideo['Data']['Wi-Fi'] * iInternet['edge'],
        'Metro': trafficDestination['Wi-Fi']['Traffic to content provider'] * iInternet['metro'],
        'Core': trafficDestination['Wi-Fi']['Traffic to content provider'] * iInternet['core'],
    }
})

# Format nicely for display
internetEnergyDisplay = pd.DataFrame({
    '4G': {
        'CPE & AN': f'{internetEnergy["4G"]["CPE & AN"].to("gigawatt hours"):,.0f~H}',
        'Edge': f'{internetEnergy["4G"]["Edge"].to("gigawatt hours"):,.0f~H}',
        'Metro': f'{internetEnergy["4G"]["Metro"].to("gigawatt hours"):,.0f~H}',
        'Core': f'{internetEnergy["4G"]["Core"].to("gigawatt hours"):,.0f~H}',
        'Total': f'{internetEnergy["4G"].sum().to("gigawatt hours"):,.0f~H}',
    },
    'Wi-Fi': {
        'CPE & AN': f'{internetEnergy["Wi-Fi"]["CPE & AN"].to("gigawatt hours"):,.0f~H}',
        'Edge': f'{internetEnergy["Wi-Fi"]["Edge"].to("gigawatt hours"):,.0f~H}',
        'Metro': f'{internetEnergy["Wi-Fi"]["Metro"].to("gigawatt hours"):,.0f~H}',
        'Core': f'{internetEnergy["Wi-Fi"]["Core"].to("gigawatt hours"):,.0f~H}',
        'Total': f'{internetEnergy["Wi-Fi"].sum().to("gigawatt hours"):,.0f~H}',
    }
})
internetEnergyDisplay

,4G,Wi-Fi
CPE & AN,"1,042 GW hr","2,803 GW hr"
Edge,9 GW hr,56 GW hr
Metro,14 GW hr,55 GW hr
Core,35 GW hr,138 GW hr
Total,"1,099 GW hr","3,052 GW hr"


## Results

We now put this all together into a concluding results statement.

### Energy consumption

In [ ]:
videoStreamingEnergy = pd.DataFrame({
    '4G': {
        'Total': internetEnergy['4G'].sum(),
        'Intensity': internetEnergy['4G'].sum() / allUKVideo['Time']['4G'],
    },
    'Wi-Fi': {
        'Total': internetEnergy['Wi-Fi'].sum(),
        'Intensity': internetEnergy['Wi-Fi'].sum() / allUKVideo['Time']['Wi-Fi'],
    },
    'Total': {
        'Total': internetEnergy['4G'].sum() + internetEnergy['Wi-Fi'].sum(),
        'Intensity': (internetEnergy['4G'].sum() + internetEnergy['Wi-Fi'].sum()) / allUKVideo['Time']['Total']
    }
})

# As a percentage of UK generation
ukGeneration2019 = 324.8 * ureg.terawatthours  # Source [20]
percentageGeneration = (videoStreamingEnergy['Total']['Total'].to(
    'gigawatthours') / ukGeneration2019) / 1000

output = (
    f'We estimate {allUKVideo["Time"]["Total"].to("hours").to_compact().magnitude:,.1f}bn '
    f'hours of video were streamed in the UK in 2019 generating '
    f'{allUKVideo["Data"]["Total"].to("exabyte"):,.1f~H} of data and consuming '
    f'a total of {videoStreamingEnergy["Total"]["Total"].to("terawatthours"):,.1f~H} of electricity, or '
    f'{percentageGeneration.magnitude:.1%} of total electricity generation. '
    f'By connection type, energy consumption was split {videoStreamingEnergy["4G"]["Total"].to("terawatthours"):,.1f~H} '
    f'over 4G and {videoStreamingEnergy["Wi-Fi"]["Total"].to("terawatthours"):,.1f~H} over Wi-Fi. '
    f'Video streaming over 4G is twice as energy intensive per streaming hour '
    f'({videoStreamingEnergy["4G"]["Intensity"].to("kilowatthours/hour"):,.3f~H}) compared to Wi-Fi '
    f'({videoStreamingEnergy["Wi-Fi"]["Intensity"].to("kilowatthours/hour"):,.3f~H}).'
)
output

'We estimate 38.9bn hours of video were streamed in the UK in 2019 generating 75.8 EB of data and consuming a total of 4.2 TWh of electricity, or 1.3% of total electricity generation. By connection type, energy consumption was split 1.1 TWh over 4G and 3.1 TWh over Wi-Fi. Video streaming over 4G is twice as energy intensive per streaming hour (0.207 kWh/hr) compared to Wi-Fi (0.091 kWh/hr).'

### Carbon footprint

Having calculated the energy intensity, we can apply the 2019 electricity carbon conversion factor to calculate the carbon intensity of an hour of video streaming.

In [ ]:
conversionFactor = 0.254 # Source [21]
carbonPerHourStreaming = videoStreamingEnergy['Total']['Intensity'].magnitude * conversionFactor
print(f'Carbon intensity of use-stage networking for video streaming: {carbonPerHourStreaming:,.3f} kgCO2/hour')

Carbon intensity of use-stage networking for video streaming: 0.027 kgCO2/hour


## References

1. UKOM (2019) Q4 2019 UK Digital Market Overview report. Available from: https://ukom.uk.net/uploads/files/news/ukom/174/UKOM_Digital_Marketing_Overview_December_2019_final.pdf

2. Ofcom (2020) Online Nation – 2020 report. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0027/196407/online-nation-2020-report.pdf

3. Solsman, J.E. (2018) Normally secretive Netflix inches back the curtain on how subscribers stream. 7 March 2018. CNET. Available from: https://www.cnet.com/news/netflix-shares-streaming-data-by-device-country-mobile-wi-fi-movies-tv/

4. Hindy, J. (2019) How much data does YouTube actually use? 30 June 2019. Android Authority. Available from: https://www.androidauthority.com/how-much-data-does-youtube-use-964560/

5. Sandvine (2020) The Mobile Internet Phenomena Report. Available from: https://www.sandvine.com/download-report-mobile-internet-phenomena-report-2020-sandvine

6. Sandvine (2019) The Global Internet Phenomena Report. Available from: https://www.sandvine.com/global-internet-phenomena-report-2019

7. Ofcom (2019) Connected Nations 2019. 20 December 2019. Ofcom. Available from: https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research/connected-nations-2019/main-report

8. Chee, F.Y. (2020) YouTube, Amazon Prime forgo streaming quality to relieve European networks. Reuters. 20 March. Available from: https://uk.reuters.com/article/us-health-coronavirus-youtube-exclusive-idUKKBN2170OP

9. Ofcom (2019) Media Nations: UK 2019. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0019/160714/media-nations-2019-uk-report.pdf

10. Office for National Statistics (2019) Overview of the UK population. 23 August 2019. Available from: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/articles/overviewoftheukpopulation/august2019

11. Coroama, V.C., Schien, D., Preist, C. & Hilty, L.M. (2015) The Energy Intensity of the Internet: Home and Access Networks. In: Lorenz M. Hilty & Bernard Aebischer (eds.). ICT Innovations for Sustainability. Advances in Intelligent Systems and Computing. 2015 Cham, Springer International Publishing. pp. 137–155. Available from: doi:10.1007/978-3-319-09228-7_8.

12. Schien, D., Coroama, V.C., Hilty, L.M. & Preist, C. (2015) The Energy Intensity of the Internet: Edge and Core Networks. In: Lorenz M. Hilty & Bernard Aebischer (eds.). ICT Innovations for Sustainability. Advances in Intelligent Systems and Computing. 2015 Cham, Springer International Publishing. pp. 157–170. Available from: doi:10.1007/978-3-319-09228-7_9.

13. Masanet, E., Shehabi, A., Lei, N., Smith, S., et al. (2020) Recalibrating global data center energy-use estimates. Science. 367 (6481), 984–986. Available from: doi:10.1126/science.aba3758.

14. Nissen, N.F. (2007) EuP Preparatory Study Lot 6 “Standby and Off-mode Losses. Available from: https://www.eup-network.de/fileadmin/user_upload/Produktgruppen/Lots/Final_Documents/Los_06_final_report.pdf

15. Pihkola, H., Hongisto, M., Apilo, O. & Lasanen, M. (2018) Evaluating the Energy Consumption of Mobile Data Transfer—From Technology Development to Consumer Behaviour and Life Cycle Thinking. Sustainability. 10 (7), 2494. Available from: doi:10.3390/su10072494.

16. Apple (2019) iPhone 11 Pro - Technical Specifications. 2019. Apple. Available from: https://www.apple.com/iphone-11-pro/specs/

17. Espósito, F. (2019) iPhone 11 battery size confirmed in new regulatory filings. 9to5Mac. Available from: https://9to5mac.com/2019/09/17/iphone-11-and-iphone-11-pro-battery-size/

18. Apple (n.d.) iPhone - Battery Test Information - Apple. Available from: https://www.apple.com/iphone/battery.html

19. Aslan, J., Mayers, K., Koomey, J.G. & France, C. (2018) Electricity Intensity of Internet Data Transmission: Untangling the Estimates: Electricity Intensity of Data Transmission. Journal of Industrial Ecology. 22 (4), 785–798. Available from: doi:10.1111/jiec.12630.

20. Department for Business, Energy & Industrial Strategy (2020) UK energy in brief 2020. 30 July 2020. GOV.UK. Available from: https://www.gov.uk/government/statistics/uk-energy-in-brief-2020

21. Department for Business, Energy & Industrial Strategy (2020) Greenhouse gas reporting: conversion factors 2019. 28 July 2020. GOV.UK. Available from: https://www.gov.uk/government/publications/greenhouse-gas-reporting-conversion-factors-2019

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b38c2c00-d173-47f7-8844-adf84ba73830' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>